In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import itertools as it

In [72]:
#data = pd.read_csv('movie_bd_v5.csv')
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
525,tt0268397,25000000,80936232,Jimmy Neutron: Boy Genius,Debi Derryberry|Megan Cavanagh|Mark DeCarlo|Je...,John A. Davis,"He may be small, but he's got a big brain!",Jimmy Neutron is a boy genius and way ahead of...,83,Adventure|Fantasy|Animation|Action|Comedy,Nickelodeon Movies|Universal Cartoon Studios|P...,12/21/2001,5.5,2001
737,tt0433035,110000000,299268508,Real Steel,Hugh Jackman|Dakota Goyo|Evangeline Lilly|Anth...,Shawn Levy,"If you get one shot, make it real.","In the near-future, Charlie Kenton is a washed...",127,Action|Science Fiction|Drama,DreamWorks SKG|21 Laps Entertainment|Reliance ...,9/28/2011,6.5,2011
391,tt0800320,125000000,232713139,Clash of the Titans,Sam Worthington|Liam Neeson|Ralph Fiennes|Gemm...,Louis Leterrier,Titans will clash.,"Born of a god but raised as a man, Perseus is ...",106,Adventure|Fantasy|Action,The Zanuck Company|Legendary Pictures|Thunder ...,4/1/2010,5.6,2010
914,tt0268126,19000000,32801173,Adaptation.,Nicolas Cage|Meryl Streep|Chris Cooper|Tilda S...,Spike Jonze,Charlie Kaufman writes the way he lives... Wit...,A love-lorn script writer grows increasingly d...,114,Comedy|Crime|Drama,Columbia Pictures|Beverly Detroit|Propaganda F...,12/6/2002,7.2,2002
1208,tt1428538,50000000,224803475,Hansel & Gretel: Witch Hunters,Jeremy Renner|Gemma Arterton|Famke Janssen|Pih...,Tommy Wirkola,Classic tale. New twist.,"After getting a taste for blood as children, H...",88,Fantasy|Horror|Action,Paramount Pictures|Studio Babelsberg|MTV Films...,1/17/2013,5.7,2013


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [73]:
answers = {} # создадим словарь для ответов


# Добработки к основной таблице данных

# Добавим новый столбец profit (прибыль), profit = revenue - budget
data['profit'] = data['revenue'] - data['budget']

# Добавим новый столбец release_month в котором будет содержаться месяц релиза
data['release_month'] = data['release_date'].apply(lambda x: int(x.split('/')[0]))

# Добавим столбец со списком причастных студий
data['production_companies_list'] = data['production_companies'].apply(lambda x: x.split('|'))

# Добавим столбец с длинами названий
data['title_lenght'] = data['original_title'].apply(lambda x: len(x))

# Добавим столбец с числом слов в описании
data['overview_word_count'] = data['overview'].apply(lambda x: len(x.split(' ')))

# Добавим столбец со списом (list) актеров 
data['cast_list'] = data['cast'].apply(lambda x: x.split('|'))


# Функции для обработки данных

# Функция series_split предназначена для разделения серий, в которых в каджом элементе (строке) 
# записано несколько значений, разделенных сепаратором. Функция возвращает серию, 
# в которой каждое отдельное значение в исходной строке является отдельной строкой.

def series_split(series,separator='|'):
    values = []
    for line in series:
        values_list = line.split(separator)
        for single_value in values_list:
            values.append(single_value)
    return(pd.Series(values))
                    

# Функция series_enumeration возвращает серию в которой перечислены все сочитания внутри каждой из строк серии
# Строки анализируемой серии должны быть списком (list) с анализируемыми занчениями
# Для работы функции необходимо импортировать бибилиотек itertools 'import itertools as it'

def series_enumeration(series_list,pairs_lenght=2):
    values = []
    for group in series_list:
        pairs = list(it.combinations(group, pairs_lenght))
        for pair in pairs:
            values.append(pair)
    return(pd.Series(values))


# Функция director_sum для учета фильмов снятых несколькими режисеррами для каждого из отдельных режисеров
# на вход подается серия с числом фильмов снятых режисерами возвращается измененная серия
# Лучше воспользоваться стандартным методом))

def directors_sum(series):
    for i in range(len(series)):
        for j in range(len(series)):
            if series.index[i] in series.index[j]:
                series.values[i] += 1
    return(series)


In [5]:
# Посмотрим, что получилось после преобразований
data.sample(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,production_companies_list,title_lenght,overview_word_count,cast_list
310,tt0962736,35000000,27409889,The Young Victoria,Emily Blunt|Jim Broadbent|Mark Strong|Paul Bet...,Jean-Marc VallÃ©e,Love rules all,From Academy AwardÂ® winners Graham King and M...,105,Drama|History|Romance,GK Films,3/4/2009,6.9,2009,-7590111,3,[GK Films],18,116,"[Emily Blunt, Jim Broadbent, Mark Strong, Paul..."


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
answers['1'] = '3. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# верно (+)

In [7]:
data[data['budget'] == data['budget'].max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

#### Вариант решения 2

In [8]:
data.sort_values('budget', ascending = False).head(1).original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
answers['2'] = '2. Gods and Generals (tt0279111)'
# верно (+)

In [10]:
data[data['runtime'] == data['runtime'].max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

#### Вариант решения 2

In [11]:
data.sort_values('runtime', ascending = False).head(1).original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
# верно (+)

In [13]:
data[data['runtime'] == data['runtime'].min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

#### Вариант решения 2

In [14]:
data.sort_values('runtime', ascending = True).head(1).original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [15]:
answers['4'] = '2. 110'
# верно (+)

In [16]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = '1. 107'
# верно (+)

In [18]:
data['runtime'].describe()["50%"]

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [69]:
answers['6'] = '5. Avatar (tt0499549)'
# верно (+)

In [20]:
data[data['profit'] == data['profit'].max()][['original_title','profit']]

,original_title,profit
239,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [21]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
# верно (+)

In [22]:
data[data['profit'] == data['profit'].min()][['original_title','profit']]

,original_title,profit
1245,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
answers['8'] = '1. 1478'
# верно (+)

In [24]:
len(data[data['profit'] > 0])

1478

#### вариант решения 2

In [25]:
len(data.query('profit > 0'))

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [26]:
answers['9'] = '4. The Dark Knight (tt0468569)'
# верно (+)

In [27]:
data_by_year = data.query('release_year == 2008')
data_by_year[data_by_year['profit'] == data_by_year['profit'].max()][['original_title','profit']]

,original_title,profit
599,The Dark Knight,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [28]:
answers['10'] = '5. The Lone Ranger (tt1210819)'
# верно (+)

In [29]:
data_by_year = data.query('release_year >= 2012 & release_year <= 2014')
data_by_year[data_by_year['profit'] == data_by_year['profit'].min()][['original_title','release_year','profit']]

,original_title,release_year,profit
1245,The Lone Ranger,2013,-165710090


# 11. Какого жанра фильмов больше всего?

In [30]:
answers['11'] = '3. Drama'
# верно (+)

In [31]:
ganres = series_split(data.genres)
display(ganres.value_counts().sort_values(ascending = False))

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

#### вариант решения 2

In [32]:
sum_genres=pd.Series((data['genres']).str.cat(sep='|').split('|')).value_counts(ascending=False)
display(sum_genres)

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = '1. Drama'
# верно (+)

In [34]:
data_by_profit = data.query('profit > 0')
series_split(data_by_profit['genres']).value_counts(ascending=False)

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

####  вариант решения 2

In [35]:
data_by_profit = data.query('profit > 0')
sum_genres = pd.Series((data_by_profit['genres']).str.cat(sep='|').split('|')).value_counts(ascending=False)
display(sum_genres)

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [36]:
answers['13'] = '5. Peter Jackson'
# верно (+)

In [37]:
grouped_data = data.groupby(['director']).sum()
grouped_data[grouped_data['revenue'] == grouped_data['revenue'].max()]

,budget,revenue,runtime,vote_average,release_year,profit,release_month,title_lenght,overview_word_count
director,,,,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685,95,265,363


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [70]:
answers['14'] = '3. Robert Rodriguez'
# верно (+)

#### Вариант 1
При итоговой проверке понял, что первый способ решения оказался достаточно "велосипедным" да и в целом выдает результаты на 1 больше, но оставлю его на память))

In [39]:
directors = data[data.genres.str.match('Action')].groupby('director')['original_title'].count().sort_values(ascending = False)
directors_summed = directors_sum(directors)
display(directors_summed.head())

director
Robert Rodriguez      8
Tony Scott            6
Antoine Fuqua         6
Andrzej Bartkowiak    6
Zack Snyder           5
Name: original_title, dtype: int64

#### Вариант 2
Понял, что все эти задачи можно также решать с помощью функции series_split

In [40]:
directors = data[data.genres.str.match('Action')]['director']
series_split(directors).value_counts().sort_values(ascending = False).head()

Robert Rodriguez      7
Tony Scott            5
Antoine Fuqua         5
Andrzej Bartkowiak    5
John Moore            4
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = '3. Chris Hemsworth'
# верно (+)

In [42]:
actors_by_revenue = data[data.release_year == 2012][['cast_list', 'revenue']].explode('cast_list')
actors_by_revenue.groupby('cast_list').sum().sort_values('revenue', ascending = False).head()

,revenue
cast_list,
Chris Hemsworth,2027450773
Denis Leary,1629460639
Anne Hathaway,1522851057
Robert Downey Jr.,1519557910
Mark Ruffalo,1519557910


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = '3. Matt Damon'
# верно (+)

In [44]:
high_budget = data[data['budget'] > data['budget'].mean()]
series_split(high_budget['cast']).value_counts().sort_values(ascending = False).head()

Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Tom Cruise           15
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = '2. Action'
# верно (+)

In [46]:
Cage_movies = data[data['cast'].str.contains('Nicolas Cage')]
series_split(Cage_movies['genres']).value_counts().sort_values(ascending = False).head()

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
# верно (+)

In [48]:
Paramount_movies = data[data['production_companies'].str.contains('Paramount Pictures')]
Paramount_movies[Paramount_movies['profit'] == Paramount_movies['profit'].min()][['original_title','profit']]

,original_title,profit
925,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '5. 2015'
# верно (+)

In [50]:
data_by_year = data.groupby('release_year').sum()
data_by_year[data_by_year['revenue'] == data_by_year['revenue'].max()]

,budget,revenue,runtime,vote_average,profit,release_month,title_lenght,overview_word_count
release_year,,,,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378,842,1601,5524


# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '1. 2014'
# верно (+)

In [52]:
WB_by_year = data[data['production_companies'].str.contains('Warner Bros')].groupby('release_year').sum()
WB_by_year[WB_by_year['profit'] == WB_by_year['profit'].max()]

,budget,revenue,runtime,vote_average,profit,release_month,title_lenght,overview_word_count
release_year,,,,,,,,
2014,947600000,3243064519,1576,88.9,2295464519,89,176,672


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = '4. Сентябрь'
# верно (+)

In [54]:
data.groupby('release_month')['original_title'].count().sort_values(ascending = False).head()

release_month
9     227
12    190
10    186
8     161
3     156
Name: original_title, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
answers['22'] = '2. 450'
# верно (+)

In [56]:
data[(data['release_month'] >= 6)&(data['release_month'] <= 8)]['imdb_id'].nunique()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
answers['23'] = '5. Peter Jackson'
# верно (+)

In [58]:
winter_movies = data[(data['release_month'] >= 12)|(data['release_month'] <= 2)]
series_split(winter_movies['director']).value_counts().sort_values(ascending = False).head()

Peter Jackson        7
Clint Eastwood       6
Steven Soderbergh    6
Nancy Meyers         4
Shawn Levy           4
dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
answers['24'] = '5. Four By Two Productions'
# верно (+)

In [60]:
expend_data = data[['production_companies_list','title_lenght']]
expend_data.explode('production_companies_list').groupby('production_companies_list').mean().sort_values('title_lenght', ascending = False).head()

,title_lenght
production_companies_list,
Four By Two Productions,83.0
"Jim Henson Company, The",59.0
Dos Corazones,47.0
Polsky Films,46.0
Museum Canada Productions,46.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
answers['25'] = '3. Midnight Picture Show'
# верно (+)

In [62]:
expend_data = data[['production_companies_list','overview_word_count']]
expend_data.explode('production_companies_list').groupby('production_companies_list').mean().sort_values('overview_word_count', ascending = False).head()

,overview_word_count
production_companies_list,
Midnight Picture Show,175.0
Heineken Branded Entertainment,161.0
Room 9 Entertainment,161.0
98 MPH Productions,159.0
Brookwell-McNamara Entertainment,156.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
# верно (+)

In [64]:
best_movies = data[['vote_average', 'original_title']].sort_values('vote_average',ascending = False)
best_movies[:round(len(best_movies)*0.01)]

,vote_average,original_title
599,8.1,The Dark Knight
118,8.0,Interstellar
125,8.0,The Imitation Game
9,8.0,Inside Out
34,8.0,Room
1183,7.9,The Wolf of Wall Street
128,7.9,Gone Girl
1191,7.9,12 Years a Slave
119,7.9,Guardians of the Galaxy
1081,7.9,The Lord of the Rings: The Return of the King


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
# верно (+)

In [66]:
series_enumeration(data['cast_list']).value_counts().sort_values(ascending = False)

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Rupert Grint, Emma Watson)            7
(Ben Stiller, Owen Wilson)             6
(Johnny Depp, Helena Bonham Carter)    6
                                      ..
(Alexander Gould, Brad Garrett)        1
(Edward Norton, Emma Stone)            1
(Emma Watson, Israel Broussard)        1
(Morgan Freeman, Matt Damon)           1
(Aly Michalka, Gaelan Connell)         1
Length: 18121, dtype: int64

# Submission

In [71]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '3. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [68]:
# и убедиться что ни чего не пропустил)
len(answers)

27